**Connect google drive**

In [5]:

!nvidia-smi

Mon Nov 27 06:32:00 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [6]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
ln: failed to create symbolic link '/mydrive/My Drive': File exists
'19103063  1rt 202 cv01  (1).docx'
'19103063  1rt 202 cv01 .docx'
 19103063_CSC283_SectionB_Finalxm.pdf
'19103063  csc391A heap 01.docx'
'19103063- csc 391  assiggnment.docx'
'19103063_CSC451_SEC B_Kabita Yesmin.pdf'
'19103063 Java Kabita Yesmin.pdf'
'19103063_kabita(Asg02)_MAT257_A (1).pdf'
'19103063_kabita(Asg02)_MAT257_A.pdf'
 19103063-kabita-Csc391A_1stterm.pdf
'19103063-kabita._csc391 A.pdf'
'19103063_kabita-Sl 07_ElpJ mid.pdf'
'19103063_Kabitayesmin -ArtSect(C) Asg 2.pdf'
'19103063_Mat-257_(A)Asg01_kabita yesmin.pdf'
 20210207_131504-COLLAGE.jpg
'Area of rectangle. java.png'
'ASG01_CSC_439(B)-19103063_kabita_.zip'
'(asg-03)19103063_kabita yesmin_Mat257(A).pdf'
 Captures.zip
 class03.zip
 Class1-Copy1.ipynb
'Class and Object 6.gslides'
 Classroom
 CNN_KOALA
 CNN_YOlO
'Colab Notebooks'


**1) Clone the Darknet**



In [7]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15833, done.
remote: Counting objects: 100% (294/294), done.
remote: Compressing objects: 100% (152/152), done.
remote: Total 15833 (delta 157), reused 219 (delta 140), pack-reused 15539
Receiving objects: 100% (15833/15833), 14.76 MiB | 2.86 MiB/s, done.
Resolving deltas: 100% (10587/10587), done.


**2) Compile Darknet using Nvidia GPU**


In [8]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
mkdir -p results
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -rdynamic -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:945:23: warning: variable ‘rgb’ set but not used []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wunused-but-set-variable-Wunused-but-set-variable]8;;]
  945 |                 float rgb[3];
      |                       ^~~
./src/image_opencv.cpp: In function ‘void cv_draw_object(image, float*, int, int, int*, float*, int*, int, char**)’:
./src/image_opencv.cpp:1443:14: warning: un

**3) Configure Darknet network for training YOLO V3**

In [9]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [10]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

In [11]:
# Create folder on google drive so that we can save there the weights
!mkdir "/mydrive/yolov3"

mkdir: cannot create directory ‘/mydrive/yolov3’: File exists


In [12]:
!echo "Koala" > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data
!mkdir data/obj

In [13]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2023-11-27 06:33:45--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  68.4MB/s    in 2.3s    

2023-11-27 06:33:48 (68.4 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



**4) Extract Images**

The images need to be inside a zip archive called "images.zip" and they need to be inside the folder "yolov3" on Google Drive

In [14]:
!unzip /mydrive/yolov3/images.zip -d data/obj

Archive:  /mydrive/yolov3/images.zip
  inflating: data/obj/kidcyber-com.au  
  inflating: data/obj/koalaclancyfoundation-org.au  
  inflating: data/obj/koalahospital-org.au  
  inflating: data/obj/learningenglish-voanews.com  
  inflating: data/obj/light_thumbnail2.png  
  inflating: data/obj/littleeinsteins-fandom.com  
  inflating: data/obj/livescience.com  
  inflating: data/obj/logan-qld-gov.au  
  inflating: data/obj/mentalfloss.com  
  inflating: data/obj/mercurynews.com  
  inflating: data/obj/midcoast-nsw-gov.au  
  inflating: data/obj/nationalgeographic.com  
  inflating: data/obj/classes.txt    
  inflating: data/obj/images (70).jpeg  
  inflating: data/obj/images (70).txt  
  inflating: data/obj/images (71).jpeg  
  inflating: data/obj/images (71).txt  
  inflating: data/obj/images (72).jpeg  
  inflating: data/obj/images (72).txt  
  inflating: data/obj/images (73).jpeg  
  inflating: data/obj/images (73).txt  
  inflating: data/obj/images (74).jpeg  
  inflating: data/obj/

In [15]:

import glob
import os
import re

txt_file_paths = glob.glob(r"data/obj/*.txt")
for i, file_path in enumerate(txt_file_paths):
    # get image size
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
            print(line)
            numbers = re.findall("[0-9.]+", line)
            print(numbers)
            if numbers:

                # Define coordinates
                text = "{} {} {} {} {}".format(0, numbers[1], numbers[2], numbers[3], numbers[4])
                text_converted.append(text)
                print(i, file_path)
                print(text)
        # Write file
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

15 0.264444 0.491111 0.484444 0.768889

['15', '0.264444', '0.491111', '0.484444', '0.768889']
0 data/obj/images (81).txt
0 0.264444 0.491111 0.484444 0.768889
15 0.572391 0.467647 0.565657 0.829412

['15', '0.572391', '0.467647', '0.565657', '0.829412']
1 data/obj/images (99).txt
0 0.572391 0.467647 0.565657 0.829412
15 0.509677 0.515432 0.903226 0.969136

['15', '0.509677', '0.515432', '0.903226', '0.969136']
2 data/obj/images (120).txt
0 0.509677 0.515432 0.903226 0.969136
15 0.743333 0.541667 0.193333 0.535714

['15', '0.743333', '0.541667', '0.193333', '0.535714']
3 data/obj/images (115).txt
0 0.743333 0.541667 0.193333 0.535714
15 0.646667 0.553571 0.233333 0.869048

['15', '0.646667', '0.553571', '0.233333', '0.869048']
3 data/obj/images (115).txt
0 0.646667 0.553571 0.233333 0.869048
15 0.391667 0.494048 0.350000 0.976190

['15', '0.391667', '0.494048', '0.350000', '0.976190']
3 data/obj/images (115).txt
0 0.391667 0.494048 0.350000 0.976190
15 0.343636 0.505464 0.665455 0.9890

In [16]:
import glob
images_list = glob.glob("data/obj/*.jpg")
print(images_list)

[]


In [17]:
#Create training.txt file
file = open("data/train.txt", "w")
file.write("\n".join(images_list))
file.close()

**6) Start the training**

In [18]:
# Start the training
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

 CUDA-version: 11080 (12000), cuDNN: 8.9.6, GPU count: 1  
 OpenCV version: 4.5.4
yolov3_training
 0 : compute_capability = 750, cudnn_half = 0, GPU: Tesla T4 
net.optimized_memory = 0 
mini_batch = 4, batch = 64, time_steps = 1, train = 1 
   layer   filters  size/strd(dil)      input                output
   0 Create CUDA-stream - 0 
 Create cudnn-handle 0 
conv     32       3 x 3/ 1    416 x 416 x   3 ->  416 x 416 x  32 0.299 BF
   1 conv     64       3 x 3/ 2    416 x 416 x  32 ->  208 x 208 x  64 1.595 BF
   2 conv     32       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  32 0.177 BF
   3 conv     64       3 x 3/ 1    208 x 208 x  32 ->  208 x 208 x  64 1.595 BF
   4 Shortcut Layer: 1,  wt = 0, wn = 0, outputs: 208 x 208 x  64 0.003 BF
   5 conv    128       3 x 3/ 2    208 x 208 x  64 ->  104 x 104 x 128 1.595 BF
   6 conv     64       1 x 1/ 1    104 x 104 x 128 ->  104 x 104 x  64 0.177 BF
   7 conv    128       3 x 3/ 1    104 x 104 x  64 ->  104 x 104 x 128 1.595 BF
   8 Sho